# Modelo con uso de `API de CSi`

## Dependecias

In [ ]:
# !pip install comtypes

In [ ]:
import os, sys, ctypes
import numpy as np
import comtypes.client

## Entrada de datos

In [ ]:
fc = 280;
material_type = 2;
material_name = f'fc={str(fc)} kg/cm^2'; # fc=280kg/cm^2
# 
E = 150000*fc*0.5; # kg/cm^2
v = 0.2
coeficienteTermico = 0.0000055 #T

# secciones para la viga y columna
secBeamb = 0.3 # cm
secBeamh = 0.6 # cm
beamName = f'V{str(secBeamb)}x{str(secBeamh)}'
secColumnb = 0.3 # cm
secColumnh = 0.6 # cm
columnName = f'C{str(secColumnb)}x{str(secColumnh)}'

pisos = 7  # numero de pisos
banos = 10
baseX, baseY, alturaZ = 3, 1, 4 # a partir del 2do piso
gridsx, gridsy = 11, 1 # grillas en X y Y

# Indicar el programa a usar
connect_to = 1; #1 para SAP2000-v26 y 2 para ETABS-v21
# Variable para definir una instancia manual para el programa y definir la ruta.
rutaEspecifica = False

### Coneccion

In [ ]:
smodel, ret = None, None
conn = {};
if connect_to == 1:  # SAP2000
    name_app="SAP2000"
    conn = {
        'app_ruta': "C:\\Program Files\\Computers and Structures\\SAP2000 26\\SAP2000.exe",
        'app_adjunto': "CSI.SAP2000.API.SapObject",
        'app_helper': 'SAP2000v1.Helper'
        }
elif (connect_to == 2): # ETABS
    name_app="ETABS"
    conn = {
        'app_ruta': "C:\\Program Files\\Computers and Structures\\ETABS 21\\ETABS.exe",
        'app_adjunto': "CSI.ETABS.API.ETABSObject",
        'app_helper': "ETABSv1.Helper"
        }
else: 
    print("No tenemos soporte aun")
    sys.exit()

In [ ]:
try:
    connect_to_app = comtypes.client.GetActiveObject(conn['app_adjunto'])
    print("Coneccion exitosa!.\nadjuntando a una instancia existente.")
    # ctypes.windll.user32.MessageBox(0,"mesnaje ...", 64) # error: 16
except (OSError, comtypes.COMError):
    print(f"No se encontró ninguna instancia en ejecución del programa({name_app}).")

    print(f"Tratando de Ejecutar {name_app}!.")
    #create API helper object
    # helper = comtypes.client.CreateObject(conn['app_helper']).QueryInterface()
    helper = comtypes.client.CreateObject(conn['app_helper'])
    if connect_to == 1:
        helper = helper.QueryInterface(comtypes.gen.SAP2000v1.cHelper)
    else:
        helper = helper.QueryInterface(comtypes.gen.ETABSv1.cHelper);
    if rutaEspecifica: # para versiones anteriores a la ultima version instalada
        ProgramPath = conn['app_ruta'];
        try:
            connect_to_app = helper.CreateObject(ProgramPath)
            print("Coneccion establecida!.\nConexion Manual")
        except (OSError, comtypes.COMError):
            print("Cannot start a new instance of the program from " + ProgramPath)
            sys.exit(-1)
    else:
        try: 
            connect_to_app = helper.CreateObjectProgID(conn['app_adjunto']) 
            print("Coneccion establecida!.")
        except (OSError, comtypes.COMError):
            print("Cannot start a new instance of the program.")
            sys.exit(-1)
    print("Ejecutando!.")
    #start ETABS application | ejecutar la Aplicacion ETABS.
    connect_to_app.ApplicationStart()

smodel = connect_to_app.SapModel;


No se encontró ninguna instancia en ejecución del programa(ETABS).
Tratando de Ejecutar ETABS!.
Coneccion establecida!.
Ejecutando!.


## Se crea un nuevo Lienzo y Modelo

In [82]:
# unidades
kN_m_C = 6
kgf_m_C = 8
Tonf_m_C = 12

In [83]:
smodel.SetModelIsLocked(False)

# Se crea un nuevo lienzo
# smodel.InitializeNewModel; # esto es valido tambien
smodel.InitializeNewModel(Tonf_m_C)

# create grid-only template model - [pisos, ]
response_status = smodel.File.NewGridOnly( 
    pisos, # numero de pisos
    baseX, alturaZ, # altura tipica de piso a partir del segundo y altura del primer nivel
    gridsx, gridsy,  # numero de grillas en X y Y
    baseX, baseY # Espaciamiento en X y Y
)

response_status

0

In [84]:
smodel.View.RefreshView(0, False);

## Se establece las propiedades del concreto

In [85]:
# definicion del material
response_status = smodel.PropMaterial.SetMaterial( material_name, material_type )
# propiedades mecanicas del material
response_status = smodel.PropMaterial.SetMPIsotropic( material_name, E, v, coeficienteTermico )
display(response_status)

0

In [86]:
# Secciones de viga y columna
response_status = smodel.PropFrame.SetRectangle(beamName, material_name, secBeamb, secBeamh) # seecion para la viga
response_status = smodel.PropFrame.SetRectangle(columnName, material_name, secColumnb, secColumnh) # seccion para la columna
display(response_status)

0

In [87]:
# # modificadores de propiedad de seccion de marco
# ModValue = [1000, 0,0,1,1,1,1,1]
# response_status = smodel.PropFrame.SetModifiers(nombreViga, ModValue)
# response_status = smodel.PropFrame.SetModifiers(nombreColumna, ModValue)

## Geometria de la estructura

`Sintax` vertice o coordenadas

En lenguaje `C#` 
```c#
int AddCartesian(
	double X,
	double Y,
	double Z,
	ref string Name,
	string UserName = "",
	string CSys = "Global",
	bool MergeOff = false,
	int MergeNumber = 0
)
```
Su equivalente en `python`
```py
AddCartesian(
	X: float,
	Y: float,
	Z: float,
	Name: str,
	UserName = "",      # referencia a un str
	CSys = "Global",    # referencia a un str
	MergeOff = false,   # referencia a un bool
	MergeNumber = 0     # referencia a un int
) -> int
```

`Sintax` Elementos o miembros de coneccion

En lenguaje `C#` 
```c#
int AddByPoint(
	string Point1,
	string Point2,
	ref string Name,
	string PropName = "Default",
	string UserName = ""
)
```
Su equivalente en `python`
```py
AddByPoint(
	Point1: str,
	Point2: str,
	Name: str, # referncia
	PropName = "Default",
	UserName = ""
) -> int
```

In [101]:
for piso in range(1, pisos+1):
    for banoCol in range(banos+1): # columnas
        FrameName = ''
        [FrameName, response_status] = smodel.FrameObj.AddByCoord(
            # idx*b, 0, 0, idx*b, 0, h, "FrameName"+str(idx+1), columnName, str(idx+1), 'Global'
            banoCol*baseX, 0, piso*alturaZ, banoCol*baseX, 0, (piso)*alturaZ, FrameName, columnName, '', 'Global'
        )
        # display(response_status)

    for banoBeam in range(banos): # vigas
        FrameName = ''
        [FrameName, response_status] = smodel.FrameObj.AddByCoord(
            #  idx*b, 0, h, (idx+1)*b, 0, h, "FrameName"+str(idx+1), beamName, str(idx+1), 'Global'
            banoBeam*baseX, 0, (piso)*alturaZ, (banoBeam+1)*baseX, 0, (piso)*alturaZ, FrameName, columnName, '', 'Global'
        )
        # display(response_status)

    # # Creacion de muros o placas de concreto tipo Shell
    # name_wall = ""
    # x = [0, baseX, 0, baseX] #b: longitud de base
    # y = [0,0,0,0]
    # z = [piso, piso, (piso+1)*alturaZ, (piso+1)*alturaZ]
    # response_status = smodel.AreaObj.AddByCoord(4, x, y, z, name_wall)
    # # diviciones en area
    
    # x = [(banos-1)*baseX, banos*baseX, banos*baseX, (banos-1)*baseX] #b: longitud de base
    # y = [0,0,0,0]
    # z = [piso, piso, (piso+1)*alturaZ, (piso+1)*alturaZ]
    # response_status = smodel.AreaObj.AddByCoord(4, x, y, z, name_wall)
    # # divisiones en area

In [102]:
smodel.View.RefreshView(0, False);

## Restricciones o condiciones de borde (apoyos)

In [90]:
restrictValues = [1, 1, 1, 1, 1, 1]

nr = banos+1
for idx in range(nr):
    node_i, node_j = ' ', ' '
    [node_i, node_j, response_status] = smodel.FrameObj.GetPoints(str(idx+1), node_i, node_j)
    response_status = smodel.Pointobj.SetRestraint(node_i, restrictValues);
    # display(response_status)

In [65]:
smodel.View.RefreshView(0, False);

## Se establece carga puntual y distribuida

In [66]:
# Establecemos patron de carga
LTYPE_OTHER = 8
nombreF = "FuerzaH"
response_status = smodel.LoadPatterns.Add(nombreF, LTYPE_OTHER, 1, True)
response_status

0

In [67]:
# Cargas puntuales
PointName1, PointName2 = ' ',' '
PointLoadValue = [1, 0, 0, 0, 0, 0]
[PointName1, PointName2, response_status] = smodel.FrameObj.GetPoints("1", PointName1, PointName2)
response_status = smodel.PointObj.SetLoadForce(PointName2, "FuerzaH", PointLoadValue)
response_status

# enBano, enPiso = 0, 0
# for piso in range(pisos):
#     [PointName1, PointName2, response_status] = smodel.FrameObj.GetPoints(str(piso+1), PointName1, PointName2)
#     # response_status = smodel.PointObj.SetLoadForce(PointName2, "FuerzaH", PointLoadValue)
#     # print(response_status)
#     # enPiso = 2*((piso+1)*banos) + (piso+1)
#     print([PointName1, PointName2, response_status])

[(1.0, 0.0, 0.0, 0.0, 0.0, 0.0), 0]

In [68]:
# Cargas distribuidas (Dead and Live)
enBano, enPiso = 0, 0
for piso in range(pisos):
    for bano in range(banos):
        enBano = ( (banos+2) + bano ) + enPiso
        response_status = smodel.FrameObj.SetLoadDistributed( str(enBano), "Live", 1, 2, 0,1, -0.25, -0.25, "Local");
        response_status = smodel.FrameObj.SetLoadDistributed( str(enBano), "Dead", 1, 2, 0,1, -0.4, -0.4, "Local");
        # display(response_status)
    enPiso = 2*((piso+1)*banos) + (piso+1)

## guardar el modelo

In [ ]:
APIPath = 'C:\\CSi_API_Example'
if not os.path.exists(APIPath):
    try: os.makedirs(APIPath)
    except OSError: pass
ModelPath = APIPath + os.sep + 'API_frame-001.edb'
#save model
smodel.File.Save(ModelPath)

## Analisis

In [18]:
smodel.Analyze.RunAnalysis();

## Calculamos resultdos

## Cerramos la coneccion

In [77]:
if (connect_to_app.ApplicationExit(False)):
    print("No se pudo cerrar")
else:
    print("Se cerro con exito.")

smodel, connect_to_app = None, None
del smodel, connect_to_app
exit(-1)

Se cerro con exito.


## Referencias.

- [1] [Archivo de video]: https://youtu.be/yxljs6hWr-o, https://youtu.be/Za4I0leC7F8, https://youtu.be/QeAdkCaRn1c